In [1]:
import os, sys, csv, gzip
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy import io

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 ## Output Type 3 (Type3) or Type 42 (TrueType)
rcParams['font.sans-serif'] = 'Arial'
%matplotlib inline

from plots import COLORS10, enlarge_tick_fontsize
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.cross_validation import (StratifiedKFold, cross_val_score)
from sklearn.metrics import *
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble
from sklearn.pipeline import Pipeline
import xgboost as xgb

# 1. Prepare the gene/feature attribute matrices and target class to for supervised machine learning

Load known Barrier Complex Genes components from file

In [186]:
df_labels = pd.read_csv('data/components.csv')
print df_labels.shape
df_labels.head()

(63, 1)


,Official Symbol
0,S100A10
1,S100A11
2,TCHHL1
3,TCHH
4,RPTN


These files below were downloaded from [Harmonizome](http://amp.pharm.mssm.edu/Harmonizome/) as the features for genes to predict whether they are adhesome components. 

There are two types datasets: continues and binary. Continues datasets were standardized.    

In [ ]:
filenames = [
        #'InterPro_gene_attribute_matrix.txt.gz', 
        'ENCODE-TF_gene_attribute_matrix.txt.gz',
        'CCLE-GE_gene_attribute_matrix_standardized.txt.gz',
        'Allen-adult-brain_gene_attribute_matrix_standardized.txt.gz',
        'Allen-dev-brain_gene_attribute_matrix_standardized.txt.gz'
    ]
basenames = [f.split('_')[0] for f in filenames]
are_continues = ['_standardized' in f for f in filenames]

In [ ]:
def read_gene_attribute_matrix(fn):
	'''
	Reads a gzipped file downloaded from Harmonizome into a pandas.DataFrame,
	with GeneSym as index. 
	'''
	with gzip.open(fn) as f:
		reader = csv.reader(f, delimiter='\t')
		header = reader.next()
		header[0] = 'GeneSym'
		# Skip line 2, 3
		reader.next()
		reader.next()

		# Remove the second and third columns which are meta data for GeneSym
		header[1:3] = []

		i = 0
		df = []
        
		for row in reader:
			row[1:3] = []
			row[1:] = map(float, row[1:])
			df.append(dict(zip(header, row)))
			i += 1
			if i % 2000 == 0:
				print i
		df = pd.DataFrame().from_records(df)[header]
		df.set_index('GeneSym', inplace=True, verify_integrity=True)

#  		if '_standardized' in fn: # continues
# 			for row in reader:
# 				row[1:3] = []
# 				row[1:] = map(float, row[1:])
# 				df.append(dict(zip(header, row)))
# 				i += 1
# 				if i % 2000 == 0:
# 					print i
# 			df = pd.DataFrame().from_records(df)[header]
# 			df.set_index('GeneSym', inplace=True, verify_integrity=True)
					
# 		else: # convert values to int and make sparse df
# 			for row in reader:
# 				row[1:3] = []
# 				row[1:] = map(lambda x: int(float(x)), row[1:])

# 				d = {h: v for h, v in zip(header, row) if v != 0}
# 				df.append(d)
# 				i += 1
# 				if i % 2000 == 0:
# 					print i
# 			df = pd.DataFrame().from_records(df)[header]\
# 				.set_index('GeneSym', verify_integrity=True)\
# 				.to_sparse(fill_value=0)
		
	return df

In [ ]:
## Parse data files downloaded from Harmonizome 
dfs = []
for fn in filenames:
    df = read_gene_attribute_matrix('data/%s' % fn)
    print fn, df.shape
    dfs.append(df)

In [ ]:
#dfs

In [ ]:
#dfs[dfs== np.nan] = str(0.0)


In [ ]:
#df_labels[y>0]

In [ ]:
## INNER JOIN all feature dfs 
df_joined = reduce(lambda a, b: pd.merge(a, b, left_index=True, right_index=True, how='outer'), 
           dfs)
print df_joined.shape

In [ ]:
df_joined.replace(np.nan, 0.0, inplace=True)
df_joined.replace(np.inf, 0.0, inplace=True)

In [ ]:
#'FLG' in df_joined.index.sort_values()

In [ ]:
## Create y 
RNG = 2016
y = np.in1d(df_joined.index, df_labels['Official Symbol']).astype(np.int8)
print y.sum()
## Make CV
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=RNG)
## Export y
df_labels = pd.DataFrame({'y': y, 'GeneSym': df_joined.index}).set_index('GeneSym')
print df_labels.shape
df_labels.to_csv('data/Barrier_Complex_Genes.csv')
df_labels.head()

In [ ]:
## Keep only the shared genes across the 4 datasets
dfs = [df.ix[df_joined.index] for df in dfs]

In [ ]:
## Export processed matrices in dfs
feature_names = {} # To store feature names
i = 0
# for basename, df in zip(basenames, dfs):
#     feature_names[basename] = df.columns.tolist()
#     if are_continues[i]:
#         np.save('data/%s_shared' % basename, df.values)
#     else: # sparse matrix
#         io.mmwrite('data/%s_shared.mtx' % basename, sp.csr_matrix(df.values))
#     i += 1

for basename, df in zip(basenames, dfs):
    feature_names[basename] = df.columns.tolist()

    np.save('data/%s_shared' % basename, df.values)


del df_joined
del dfs, df

# 2. Load preprocessed matrices and perform classifications

In [ ]:
# Load from matrices files generated above
Xs = []
# for i, basename in enumerate(basenames):
#     if are_continues[i]:
#         Xs.append(np.load('data/%s_shared.npy' % basename))
#     else:
#         Xs.append(io.mmread('data/%s_shared.mtx' % basename))#.tocsr())

for i, basename in enumerate(basenames):
    Xs.append(np.load('data/%s_shared.npy' % basename))

df_labels = pd.read_csv('data/Barrier_Complex_Genes.csv')
y = df_labels['y'].values
ratio = float(np.sum(y == 0)) / np.sum(y==1)
print 'Number of known P53 components: %d' % y.sum()
print 'Ratio of negative labels over positive labels: %.4f' % ratio

Perform dimentionality reduction using `TruncatedSVD` for all the matrices

In [ ]:
#Xs[1] = np.nan_to_num(Xs[1])

In [ ]:
Xs

In [ ]:
all_loadings = [] # collect loading matrices from SVD
for i, basename in enumerate(basenames):
    print i
    svd = TruncatedSVD(n_components=60, random_state=RNG)
    Xs[i] = np.nan_to_num(Xs[i])
    Xs[i] = svd.fit_transform(Xs[i])
    all_loadings.append(svd.components_)    

X_combined = np.hstack(Xs)

In [ ]:
## Helper functions for evaluating classifiers
def cross_val_predictions(est, X, y, cv):
	'''to get out-of-sample predictions and scores'''
	y_preds = np.zeros(y.shape)
	y_probas = np.zeros(y.shape)
	for train_idx, valid_idx in cv:
		print X[train_idx].shape, y[train_idx].shape
		est.fit(X[train_idx], y[train_idx])
		y_preds[valid_idx] = est.predict(X[valid_idx])
		y_probas[valid_idx] = est.predict_proba(X[valid_idx])[:,1]
	return y_preds, y_probas


def plot_roc(ests, Xs, y, cv, ax, colors=None, labels=None):
	all_labels = []
	total = len(labels)

	if type(ests) == list and type(Xs) != list:
		total = len(ests)
		Xs = [Xs]*total
	elif type(ests) != list and type(Xs) == list:
		ests = [ests]*total
	
	for i in range(total):
		X = Xs[i]
		est = ests[i]
		
		label = labels[i]
		color = colors[i]
		all_labels.extend([label] * len(cv))

		y_preds, y_probas = cross_val_predictions(est, X, y, cv)
		fpr, tpr, _ = roc_curve(y, y_probas)
		score = auc(fpr, tpr)
		ax.plot(fpr, tpr, label=label + ' (AUC=%.3f)' % score, color=color, lw=2)

	ax.set_xlabel('False Positive Rate', fontsize=16)
	ax.set_ylabel('True Positive Rate', fontsize=16)

	enlarge_tick_fontsize(ax, 12)
	ax.legend(loc='lower right')
	return

Estimate the number of rounds of boosting using early stopping for a single feature matrix InterPro: The boosting classifier will stop if the validation score does not improve in 50 rounds.

In [ ]:
dtrain = xgb.DMatrix(Xs[0], label=y)

param = {
    'max_depth':10, 'eta':0.05, 'silent':1, 'objective':'binary:logistic',
    'subsample': 0.4, 'colsample_bytree': 0.6,
    'min_child_weight': 50,
    'scale_pos_weight': ratio,
    'nthread': 6
}

num_round = 5000
scores = xgb.cv(param, dtrain, num_round, early_stopping_rounds=50, metrics='auc', seed=RNG, verbose_eval=10)

print scores.tail()


Estimate the number of rounds of boosting using early stopping for the combined feature matrix.

In [ ]:
dtrain = xgb.DMatrix(X_combined, label=y)

param = {
    'max_depth':12, 'eta':0.05, 'silent':1, 'objective':'binary:logistic',
    'subsample': 0.4, 'colsample_bytree': 0.4,
    'min_child_weight': 50,
    'scale_pos_weight': ratio,
    'nthread': 6
}

num_round = 5000
scores = xgb.cv(param, dtrain, num_round,
    early_stopping_rounds=50,
    metrics='auc', seed=RNG,
    verbose_eval=10)

print scores.tail()


Plot the ROC curves to evaluate the predictive performance of the GBM classifiers

In [ ]:
Xs.append(X_combined)
basenames.append('Combined')

# optimized GBM classifiers
xgbc = xgb.XGBClassifier(seed=RNG, n_estimators=39, learning_rate=0.05,
    max_depth=10, colsample_bytree=0.6, subsample=0.4, min_child_weight=50,
    gamma=0, max_delta_step=0, nthread=6, silent=True, scale_pos_weight=ratio)

xgbc2 = xgb.XGBClassifier(seed=RNG, n_estimators=159, learning_rate=0.05,
    max_depth=12, colsample_bytree=0.4, subsample=0.4, min_child_weight=50,
    gamma=0, max_delta_step=0, nthread=6, silent=True, scale_pos_weight=ratio)

clfs = [xgbc] * 5 + [xgbc2]

fig, ax = plt.subplots(figsize=(6,6))
plot_roc(clfs, Xs, y, cv, ax, colors=COLORS10, labels=basenames)

# 3. Apply the GBM classifier to all the datasets to generate predictions 

In [ ]:
RNG = 20160628
xgbc2 = xgb.XGBClassifier(seed=RNG, n_estimators=160, learning_rate=0.05,
    max_depth=12, colsample_bytree=0.4, subsample=0.4, min_child_weight=50,
    gamma=0, max_delta_step=0, nthread=8, silent=True, scale_pos_weight=ratio)

cv = StratifiedKFold(y, n_folds=10, shuffle=True, random_state=RNG)

## Get out-of-fold predictions
y_preds, y_probas = cross_val_predictions(xgbc2, X_combined, y, cv)

## Get predictions on training fold
xgbc2.fit(X_combined, y)
y_probas_on_train = xgbc2.predict_proba(X_combined)[:, 1]


In [ ]:
df_labels['OOF_preds'] = y_preds
df_labels['OOF_probas'] = y_probas
df_labels['train_probas'] = y_probas_on_train

df_labels[df_labels['OOF_preds'] != 0].sort_values('OOF_probas', ascending=False).to_csv('Results', index=False, sep='\t')
#df_labels.sort_values('OOF_preds', ascending=False)

In [ ]:
df_labels.to_csv('Barrier_Complex_Genes_on_combined_predictions.csv')

In [ ]:
df_labels[df_labels['OOF_preds'] != 0].sort_values('OOF_probas', ascending=False)

# 4. Interpret the classifier by feature importance

In [ ]:
## Get the feature_importances_ from the fitted GBM
feature_importances = xgbc2.feature_importances_
print feature_importances.shape

In [ ]:
## Count the number of original features
n_features = sum(map(len, feature_names.values()))
print 'There are %d features across these datasets used for the prediction' % n_features

Map feature importances on the SVD components back to original feature space by dot product between the feature importance vector and the loading matrix:

In [ ]:
all_feature_names = []
all_feature_fis = []
datasets = []
for i, basename in enumerate(basenames[:-1]):
    fi = feature_importances[i*60:(i+1)*60]
    loadings = all_loadings[i]
    original_feature_fis = np.dot(fi, loadings)
    original_feature_names = feature_names[basename]
    
    datasets.extend([basename] * len(original_feature_names))
    all_feature_fis.extend( original_feature_fis.tolist() )
    all_feature_names.extend( original_feature_names )

# Create a DataFrame of feature importances
df_feature_importances = pd.DataFrame({
        'dataset': datasets,
        'feature': all_feature_names, 
        'feature_importance': all_feature_fis})    

Examine the most predictive features for adhesome components:

In [ ]:
df_feature_importances.sort('feature_importance', ascending=False).head(20)

Examine the most predictive features in ENCODE-TF dataset for adhesome components:

In [ ]:
cancer_import = df_feature_importances.query('dataset == "CCLE-GE"').sort('feature_importance', ascending=False).head(10)

In [ ]:
cancer_import

Write the full feature importance table to a file and provide a link to this file.

In [ ]:
df_feature_importances.to_csv('feature_importances.csv')

from IPython.display import FileLink
FileLink('feature_importances.csv')

In [ ]:
metadata = pd.read_csv('attribute_list_entries.txt', sep='\t')

In [ ]:
metadata.set_index('CellLine', inplace=True)
metadata.head()

In [ ]:
cancer_import.insert(2, 'Tissue', np.nan)

In [ ]:
cancer_import.set_index('feature', inplace=True)
for cell in cancer_import.index:
    for comp in metadata.index:
        if cell == comp:
            cancer_import.ix[cell, 'Tissue'] = metadata.ix[comp, 'Tissue']

In [ ]:
cancer_import.reset_index(inplace=True)

Tabulate top values of cancer cells that contribute to prediction

In [ ]:
cancer_import

### Gene expresion analysis

In [ ]:
ccle_data = pd.read_csv('CCLE_Expression_Entrez_2012-09-29.gct', sep='\t', skiprows=2)

In [ ]:
ccle_data.pop('Name');

In [ ]:
ccle_data.set_index('Description', inplace=True)

In [ ]:
ccle_data = ccle_data.apply(lambda x: np.log2(x))

In [ ]:
# z-score standardize the data
ccle_data = ccle_data.apply(lambda x: (x-x.mean())/x.std(ddof=0), axis=1)

In [ ]:
ccle_data = ccle_data.T

In [ ]:
test = pd.read_csv('data/components.csv')
test = test.values.flatten().tolist()
col = ccle_data.columns.intersection(test)

In [ ]:
ccle_data = ccle_data[col]

Here List of the cancers that show the genes are found to be up (or down) regulated was compiled by method of a absolute mean. The mean and absolute mean for the gene values for each cancer sample were computed. The cancer samples were then sorted based on the absolute mean as a measure of gene presence. List were then compiled for the top 100 cancer samples that showed the highest absolute mean; if the ratio of positive to negative values was greater the sample was listed as having the genes present as up regulaors.

In [ ]:
st = ccle_data.T.describe()
st

In [ ]:
ccle_data_ab = ccle_data.apply(lambda x: abs(x))

In [ ]:
ccle_data_ab.head()

In [ ]:
abst = ccle_data_ab.T.describe()
abst

In [ ]:
meta = pd.read_json('CCLE_CL_meta.json')

In [ ]:
stats = pd.DataFrame(index=ccle_data.index, columns=['hist','hist_sub', 'gender','mean','abs_mean', 'std', 'up_reg', 'dn_reg', 'sum', 'sum_abs'])

for can in stats.index:
    stats.ix[can, 'mean'] = st.ix['mean', can]
    stats.ix[can, 'std'] = st.ix['std', can]
    stats.ix[can, 'abs_mean'] = abst.ix['mean', can]
    pos = 0
    neg = 0
    for gene in ccle_data.columns:
        if ccle_data.ix[can, gene] > 0:
            pos += 1
        elif ccle_data.ix[can, gene] < 0:
            neg += 1
    stats.ix[can, 'up_reg'] = pos
    stats.ix[can, 'dn_reg'] = neg
    for comp in meta.columns:
        if can == comp:
            stats.ix[can, 'hist'] = meta.ix['hist', comp]
            stats.ix[can, 'hist_sub'] = meta.ix['hist_sub', comp]
            stats.ix[can, 'gender'] = meta.ix['gender', comp]
    stats.ix[can, 'sum'] = ccle_data.ix[can].sum()
    stats.ix[can, 'sum_abs'] = ccle_data_ab.ix[can].sum()


stats.head()

In [ ]:
stats.sort_values(['abs_mean'], inplace=True, ascending=False)
stats.head(100)

In [ ]:
up_lst = []
dn_lst = []
for can in stats.index:
    if stats.ix[can, 'up_reg'] > stats.ix[can, 'dn_reg']:
        up_lst.append(can)
    elif stats.ix[can, 'up_reg'] < stats.ix[can, 'dn_reg']:
        dn_lst.append(can)

In [ ]:
up_lst = up_lst[0:100]
dn_lst = dn_lst[0:100]

List of cancer samples with gene presence as up regulated

In [ ]:
up_lst

List of cancer samples with gene presence as down regulated

In [ ]:
dn_lst

In [ ]:
ccle_data.ix[dn_lst]

Here List of the cancers that show the genes are found to be up (or down) regulated was compiled by method of a tertiary matrix. The data was first converted to a tertiary matrix by the top 10% of values being represented either as a 1 for up regulation of -1 for down. The values were then sumed across each cancer sample and a list of the top 100 cancers showing values were created. 

In [ ]:
vals = abs(ccle_data.values.flatten())
vals = np.sort(vals)

In [ ]:
pos = abs(ccle_data) > vals[-int(0.1*ccle_data.values.size):][0] 
up = ccle_data > 0
down = ccle_data < 0

In [ ]:
terup = pos & up
terup = terup.applymap(lambda x: 1 if x else np.nan)
terdown = pos & down
terdown = terdown.applymap(lambda x: -1 if x else np.nan)
terup.fillna(0, inplace=True)
terdown.fillna(0, inplace=True)

In [ ]:
tertiary_df = terup + terdown

In [ ]:
tertiary_df.replace(0.0, np.nan, inplace=True)
tertiary_df.dropna(axis=1, how='all', inplace=True)
tertiary_df.dropna(axis=0, how='all', inplace=True)
tertiary_df.replace(np.nan, 0, inplace=True)

In [ ]:
s = tertiary_df.sum(axis=1)

In [ ]:
s.sort_values(inplace=True)

In [ ]:
dn_lst_ter = s[0:100]
up_lst_ter = s[-100:]

In [ ]:
up_lst_ter.sort_values(inplace=True, ascending=False)


List of cancer samples with gene presence as up regulated

In [ ]:
up_lst_ter.index.tolist()

List of cancer samples with gene presence as down regulated

In [ ]:
dn_lst_ter.index.tolist()

## The Cancer Genome Atlas

In [370]:
TCGA = pd.DataFrame()

print TCGA.shape
          
for fn in os.listdir('Piccolo'):
    if 'TCGA' in fn and 'Meta' not in fn and 'MAP' not in fn:
        df = pd.read_csv('Piccolo/'+fn, sep='\t', index_col=0)
        
        lst =[]
        for a in df.index:
             lst.append(a.split('|')[5])

        df.index = lst

        df.reset_index(inplace=True)
        df.drop_duplicates(subset='index', inplace=True)
        df.set_index('index', inplace=True)

        q = df.index.intersection(df_labels.values.flatten().tolist())
        df = df.T
        df = df[q]
        df = df.T
        
        if TCGA.empty:
            TCGA = df.copy()
        else:
            TCGA = pd.concat([TCGA, df], axis=1)
        print TCGA.shape

(0, 0)
(62, 79)
(62, 512)
(62, 1768)
(62, 2077)
(62, 2122)
(62, 2668)
(62, 2716)
(62, 2914)
(62, 3089)
(62, 3655)
(62, 3746)
(62, 4364)
(62, 4687)
(62, 4866)
(62, 5400)
(62, 5824)
(62, 6425)
(62, 6980)
(62, 7067)
(62, 7497)
(62, 7680)
(62, 7867)
(62, 8425)
(62, 8602)
(62, 8867)
(62, 9340)
(62, 9793)
(62, 9949)
(62, 10521)
(62, 10643)
(62, 11236)
(62, 11293)
(62, 11373)


In [371]:
TCGA.to_csv('TCGA_tpm.csv', sep='\t')

In [472]:
TCGA = pd.read_csv('TCGA_tpm.csv', sep='\t', index_col=0)

In [473]:
TCGA = TCGA.T

In [474]:
meta = pd.read_csv('Piccolo/TCGA_Metadata.csv', sep=',')

In [475]:
meta = meta[['a_CGHubAnalysisID', 'b_disease']]

In [476]:
meta.set_index('a_CGHubAnalysisID', inplace=True)

In [477]:
#meta.head()

In [478]:
TCGA = pd.concat([TCGA, meta], axis=1)

In [479]:
TCGA.set_index('b_disease', inplace=True)

In [480]:
TCGA.index.name = 'Cancer'

In [487]:
TCGA.replace(0.0, np.nan, inplace=True)
TCGA.dropna(axis=0, thresh=int(0.9*len(TCGA.columns)), inplace=True)#drop any cancer that does not have more then 10% of these genes

In [488]:
cancers = TCGA.index.unique().tolist()

In [489]:
cancers

['HNSC', 'CESC', 'ESCA', 'BRCA', 'BLCA', 'LUSC', 'STAD', 'SKCM', 'TGCT']

In [490]:
acro = pd.read_csv('Table_of_Cancer_Acro.csv', sep='\t', index_col=0)

In [491]:
lst_can = []

for c in cancers:
    for d in acro.index:
        if c in d:
            lst_can.append(d)

Here is a list of cancers that contain at least 90% of the set of genes

In [492]:
for a in lst_can: print a

Head and Neck squamous cell carcinoma [HNSC]
Cervical squamous cell carcinoma and endocervical adenocarcinoma [CESC]
Esophageal carcinoma [ESCA]
Breast invasive carcinoma [BRCA]
Bladder Urothelial Carcinoma [BLCA]
Lung squamous cell carcinoma [LUSC]
Stomach adenocarcinoma [STAD]
Skin Cutaneous Melanoma [SKCM]
Testicular Germ Cell Tumors [TGCT]
